<a href="https://colab.research.google.com/github/kalyan2713/python_code/blob/main/PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

dataset = pd.read_csv('/content/hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
dataset.shape

(5242, 3)

In [7]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [8]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [9]:
import re
def clean_text(text):
    text=re.sub(r'[^a-zA-Z\']', ' ',text)
    text=re.sub(r'[^\x00-\x7F]+',' ' ,text)
    text=text.lower()
    return text

In [10]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [11]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i'm it's so gr...


In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
len(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [13]:
stop = stopwords.words('english')

In [14]:
def gen_freq(text):
    word_list = []
    for word in text.split():
        word_list.extend(word)
    word_freq = pd.Series(word_list).value_counts()
    word_freq = word_freq.drop(stop, error='ignore')
    return word_freq

In [15]:
def any_neg(words):
  for word in words:
    if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
      return 1
    else:
      return 0

In [16]:
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

In [17]:
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who']:
      return 1
    else:
      return 0


In [20]:
def gen_freq(text):
    word_list = []
    for word in text.split():
        word_list.extend(word)
    word_freq = pd.Series(word_list).value_counts()
    # Instead of error='ignore', use the isin() method and the '~' operator to achieve the desired effect
    word_freq = word_freq[~word_freq.index.isin(stop)]
    return word_freq

In [26]:
from sklearn.model_selection import train_test_split
x = dataset[['word_count', 'any_neg']]
y = dataset.label
X_train , X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [27]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model = model.fit(X_train, y_train)
pred = model.predict(X_test)

In [28]:
model.predict(X_test[5:10])

array([0, 0, 1, 1, 0])

In [29]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred)*100)

56.7206863679695


In [30]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
rf_pred = clf_rf.predict(X_test).astype(int)

In [31]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))
print(accuracy_score(y_test, rf_pred)*100)

[[485 114]
 [324 126]]
              precision    recall  f1-score   support

           0       0.60      0.81      0.69       599
           1       0.53      0.28      0.37       450

    accuracy                           0.58      1049
   macro avg       0.56      0.54      0.53      1049
weighted avg       0.57      0.58      0.55      1049

58.245948522402294


In [32]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(class_weight='balanced')
clf_lr.fit(X_train, y_train)
lr_pred = clf_lr.predict(X_test).astype(int)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

           0       0.62      0.61      0.61       599
           1       0.49      0.50      0.49       450

    accuracy                           0.56      1049
   macro avg       0.55      0.55      0.55      1049
weighted avg       0.56      0.56      0.56      1049

